# Module 4 Homework

## More Walmart Cleaning

The file **Walmart_United_States\_&\_Canada.csv** contains data on all
Walmarts, Sam's Clubs and Murphy USA gas/diesel in the USA and Canada.
Note that Gas/Diesel and No Over Night Parking (NOP) are indicated if
known. These data can be obtained from the site
<http://www.poi-factory.com/node/25560>.


**Before you start.** You started cleaning this data set in Activity 4.1.  Start by copying over your code and fixing the encoding issue.

In [213]:
# Copy your code from Activity 4.1 here
import pandas as pd
pd.set_option("display.max_colwidth", None) # don't cut off text
pd.set_option("display.max_columns", None) # horizontal scroll columns instead of wrapping around
from dfply import *

walmart = pd.read_csv("./data/Walmart_United_States_&_Canada.csv", names=["Long","Lat","Description","Address_Phone"])
walmart.sample(5)


,Long,Lat,Description,Address_Phone
3386,-90.282685,38.766003,"Walmart Supercenter; #1265,","10741 W Florissant; I-270 Exit 29,St Louis,MO,63136 ,,(314) 521-3422"
1445,-87.277711,30.437667,"Walmart Supercenter; #1605,Gas/Diesel,","4600 Mobile Hwy,Pensacola,FL,32506 ,,(850) 455-4320"
1583,-82.190612,28.269481,"Murphy: USA; #6836,Gas/Diesel,","7623 Gall Blvd,Zephyrhills,FL,33541 ,,(813) 715-9797"
5250,-81.100361,33.980852,"Walmart Supercenter; #1183,Gas,","2401 Augusta Rd W,West Columbia,SC,29169 ,,(803) 796-9144"
3786,-79.420006,35.733128,"Walmart Supercenter; #2845,","14215 Hwy 64 W,Siler City,NC,27344 ,,(919) 663-6000"


In [3]:
from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter

In [4]:
# View cell
(walmart
 >> select(X.Description)
 >> filter_by(~text_filter(X.Description,'Gas/Diesel', regex=True))
 >> filter_by(~text_filter(X.Description,'Gas', regex=True))
 >> filter_by(~text_filter(X.Description,'Walmart( .*)?(;|,)\s?#\d{4}', regex=True))
 >> filter_by(~text_filter(X.Description,"(Murphy|Wm|Sam's)", regex=True))
 >> filter_by(~text_filter(X.Description,"; Supercenter", regex=True))
)

/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Description


In [18]:
# Transform cell
(walmart
    >> select(X.Description)
    >> mutate(Description = ifelse(X.Description.str.contains(" ; Supercenter"),
                                  X.Descrption.str.replace(';',''), X.Description))
    >> mutate(Description = X.Description.str.replace(";",","))
    >> mutate(Store_type = X.Description.str.split(',').str.get(0),
              Store_number = X.Description.str.split(',').str.get(1),
              has_gas = ifelse(text_filter(X.Description, 'Gas'), 1, 0),
              has_diesel = ifelse(text_filter(X.Description, 'Gas/Diesel'), 1, 0))
    >> mutate(Store_number = X.Store_number.str.replace("#",""))
    >> drop(X.Description)
)

,Store_type,Store_number,has_gas,has_diesel
0,Walmart Supercentre,1050,0,0
1,Walmart Supercentre,3658,0,0
2,Walmart Supercentre,3013,0,0
3,Walmart Supercentre,3009,1,0
4,Walmart,1144,0,0
...,...,...,...,...
6811,Walmart Supercenter,4471,1,0
6812,Walmart Supercenter,1457,0,0
6813,Walmart Supercenter,1461,0,0
6814,Walmart Supercenter,1508,0,0


1.  Some of the address columns contain `(NOP)` to indicated *No overnight parking*.  Extract this information into a new indicator column, then remove it from the address column.

In [55]:
# View cell
(walmart
 >> select(X.Address_Phone)
 >> filter_by(text_filter(X.Address_Phone,'NOP', regex=True))
).head()

,Address_Phone
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
7,"3800 Memorial Dr NE,Calgary ,(NOP),AB T2A 2K2,..."
9,"5005 Northland Dr NW,Calgary ,(NOP),AB T2L 2K1..."
10,"7979 11 St SE,Calgary ,(NOP),AB T2H 0B8,(403) ..."


In [101]:
#Transform cell
(walmart
    >> select(X.Address_Phone)
    >> mutate(NOP = ifelse(X.Address_Phone.str.contains('NOP'), 1, 0))
    >> mutate(Address_Phone = ifelse(X.Address_Phone.str.contains('NOP'),
                                     X.Address_Phone.str.replace(',(NOP)','', regex=False),
                                     X.Address_Phone))
)

,Address_Phone,NOP
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295",0
1,"917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",0
2,"1110 57th Ave NE,Calgary ,AB T2E 9B7,(403) 730-0990",1
3,"1212 37 St SW,Calgary ,AB T3C 1S3,(403) 242-2205",1
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638",0
...,...,...
6811,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301 ,,(307) 417-3001",0
6812,"1733 N Federal Blvd,Riverton,WY,82501 ,,(307) 856-3261",0
6813,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901 ,,(307) 362-1957",0
6814,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801 ,(307) 674-6492",1


2.  The address column contains the phone number of most of the stores.  Extract this information into a new column.  There are some issue with different patterns, so use the divide-and-conquer approach is advised.

In [169]:
# View cell
(walmart
 >> select(X.Address_Phone)
 >> filter_by(~text_filter(X.Address_Phone,'\(?\d{3} ?(\)|-|_|\()? ?\d{3}( |-)?\d{4}', regex=True))
)

/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Address_Phone
354,"8303 Rogers Ave,Fort Smith,AR,72903 ,,(479) 452-161"
6135,"6040 Coit Rd,Plano,TX,75023,"
6193,"3440 S Bryant Blvd,San Angelo,TX,76903 ,,(325) 26-6599"


In [174]:
walmart.Address_Phone.iloc[3994]

#This is weird case has the phone number before the NOP portion, but it is going to be ignored because NOP
#will be removed so it will not matter since the phone number will be the last item in the "list" after splitting in the comma

'950 Rte 37 W,Toms River,NJ,08755,(732_349-6000,(NOP)'

In [173]:
#Transform cell
(walmart
 >> select(X.Address_Phone)
 >> mutate(PhoneNumber = case_when((text_filter(X.Address_Phone,'\(?\d{3} ?(\)|-|_|\()? ?\d{3}( |-)?\d{4}', regex=True),
                                       X.Address_Phone.str.split(",").str.get(-1).str.replace("\(|\)| |-|_","", regex=True)),
                                   (True, "Missing/Invalid")
                                  )
          )
)

#All phone numbers are normalized by removing non digit characters

/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/vector.py:110: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  return coalescer.lookup(np.arange(coalescer.shape[0]), min_nonna)


,Address_Phone,PhoneNumber
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295",4039451295
1,"917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",4037932111
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(403) 730-0990",4037300990
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) 242-2205",4032422205
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638",4032256638
...,...,...
6811,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301 ,,(307) 417-3001",3074173001
6812,"1733 N Federal Blvd,Riverton,WY,82501 ,,(307) 856-3261",3078563261
6813,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901 ,,(307) 362-1957",3073621957
6814,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801 ,(NOP),(307) 674-6492",3076746492


3.  Extract the country from the address column.

In [198]:
# View cell
(walmart
 >> select(X.Address_Phone)
 >> filter_by(~text_filter(X.Address_Phone,',[A-Z]{2},', regex=True)) #US Zipcodes
 >> filter_by(text_filter(X.Address_Phone,'[A-Z]{2}', regex=True)) #CAN Zipcodes
)

,Address_Phone
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295"
1,"917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(403) 730-0990"
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) 242-2205"
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638"
...,...
5279,"3035 Clarence Ave S,Saskatoon ,(NOP),SK S7T 0B6,(306) 653-8200"
5280,"1800 22nd Ave,Swift Current ,SK S9H 0E5,(306) 778-3489"
5281,"1000 Sims Ave,Weyburn ,SK S4H 3N9,(306) 842-6030"
5282,"240 Hamilton Rd,Yorkton ,SK S3N 4C6,(306) 782-9820"


In [199]:
#Transform cell
(walmart
    >> select(X.Address_Phone)
    >> mutate(Country = ifelse(X.Address_Phone.str.contains(',[A-Z]{2},'),"USA","Canada"))
)


,Address_Phone,Country
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295",Canada
1,"917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Canada
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(403) 730-0990",Canada
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) 242-2205",Canada
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638",Canada
...,...,...
6811,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301 ,,(307) 417-3001",USA
6812,"1733 N Federal Blvd,Riverton,WY,82501 ,,(307) 856-3261",USA
6813,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901 ,,(307) 362-1957",USA
6814,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801 ,(NOP),(307) 674-6492",USA


4.  Extract the state or province from the address columns

In [206]:
# Your code here
(walmart
    >> select(X.Address_Phone)
    >> mutate(State0Province = X.Address_Phone.str.extract(",([A-Z]{2})"))
).sample(10)

,Address_Phone,State0Province
1061,"255 W Main St,Avon,CT,06001 ,(NOP),(860) 409-0404",CT
6593,"762 Supermall Way SW,Auburn,WA,98001 ,(NOP),(253) 735-1855",WA
5177,"900 Hwy 52,Lake City,SC,29560 ,(NOP),(843) 394-7405",SC
4350,"2687 N Memorial Dr,Lancaster,OH,43130 ,,(740) 687-0323",OH
2104,"9450 W Joliet Rd,Hodgkins,IL,60525 ,(NOP),(708) 387-2090",IL
3078,"1237 32nd St; I-69 Exit 199,Port Huron,MI,48060 ,,(810) 984-5355",MI
3744,"1001 Shiloh Glen Dr; I-40 Exit 283,Morrisville,NC,27560 ,,(919) 474-2285",NC
2777,"59690 Belleview Dr,Plaquemine,LA,70764 ,,(225) 687-2550",LA
2990,"5323 E Court St N; I-69 Exit 141,Burton,MI,48509 ,(NOP),(810) 744-9690",MI
4732,"1305 Lawrence Ave W,North York ,(NOP),ON M6L 1A5,(416) 244-1171",ON


In [215]:
walmart

,Long,Lat,Description,Address_Phone
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295"
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(403) 730-0990"
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) 242-2205"
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638"
...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter; #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301 ,,(307) 417-3001"
6812,-108.379227,43.042858,"Walmart Supercenter; #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,(307) 856-3261"
6813,-109.251020,41.579761,"Walmart Supercenter; #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901 ,,(307) 362-1957"
6814,-106.940967,44.779474,"Walmart Supercenter; #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801 ,(NOP),(307) 674-6492"


5. Combine all of your transformations into one pipe, then re-factor your code to be more readable.

In [286]:
# One pipe for all transformations.
(walmart
    >> mutate(Description = ifelse(X.Description.str.contains(" ; Supercenter"),
                                  X.Descrption.str.replace(';',''), X.Description))
    >> mutate(Description = X.Description.str.replace(";",","))
    >> mutate(Store_type = X.Description.str.split(',').str.get(0),
              Store_number = X.Description.str.split(',').str.get(1),
              has_gas = ifelse(text_filter(X.Description, 'Gas'), 1, 0),
              has_diesel = ifelse(text_filter(X.Description, 'Gas/Diesel'), 1, 0))
    >> mutate(Store_number = X.Store_number.str.replace("#",""),
              NOP = ifelse(X.Address_Phone.str.contains('NOP'), 1, 0))
    >> mutate(Address_Phone = ifelse(X.Address_Phone.str.contains('NOP'),
                                     X.Address_Phone.str.replace(',(NOP)','', regex=False),
                                     X.Address_Phone),
              PhoneNumber = case_when((text_filter(X.Address_Phone,'\(?\d{3} ?(\)|-|_|\()? ?\d{3}( |-)?\d{4}', regex=True),
                                       X.Address_Phone.str.split(",").str.get(-1).str.replace("\(|\)| |-|_","", regex=True)),
                                      (True, "Missing/Invalid")),
              Country = ifelse(X.Address_Phone.str.contains(',[A-Z]{2},'),"USA","Canada"),
              State0Province = X.Address_Phone.str.extract(",([A-Z]{2})"))
)

/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/vector.py:110: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  return coalescer.lookup(np.arange(coalescer.shape[0]), min_nonna)


,Long,Lat,Description,Address_Phone,Store_type,Store_number,has_gas,has_diesel,NOP,PhoneNumber,Country,State0Province
0,-114.005671,51.262567,"Walmart Supercentre, #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295",Walmart Supercentre,1050,0,0,0,4039451295,Canada,AB
1,-111.900542,50.577939,"Walmart Supercentre, #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Walmart Supercentre,3658,0,0,0,4037932111,Canada,AB
2,-114.039133,51.107253,"Walmart Supercentre, #3013,","1110 57th Ave NE,Calgary ,AB T2E 9B7,(403) 730-0990",Walmart Supercentre,3013,0,0,1,4037300990,Canada,AB
3,-114.138488,51.040871,"Walmart Supercentre, #3009,Gas,","1212 37 St SW,Calgary ,AB T3C 1S3,(403) 242-2205",Walmart Supercentre,3009,1,0,1,4032422205,Canada,AB
4,-114.028603,50.930551,"Walmart, #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638",Walmart,1144,0,0,0,4032256638,Canada,AB
...,...,...,...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter, #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301 ,,(307) 417-3001",Walmart Supercenter,4471,1,0,0,3074173001,USA,WY
6812,-108.379227,43.042858,"Walmart Supercenter, #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,(307) 856-3261",Walmart Supercenter,1457,0,0,0,3078563261,USA,WY
6813,-109.251020,41.579761,"Walmart Supercenter, #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901 ,,(307) 362-1957",Walmart Supercenter,1461,0,0,0,3073621957,USA,WY
6814,-106.940967,44.779474,"Walmart Supercenter, #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801 ,(307) 674-6492",Walmart Supercenter,1508,0,0,1,3076746492,USA,WY


In [283]:
# Refactored coded here (definitions/lambdas then the pipe)
split_and_get = lambda col, sub, i: X[col].str.split(sub).str.get(i)
recode_ifelse = lambda col,sub,r,true,false: ifelse(text_filter(X[col], sub, regex=r), true, false)
remove_sub_if_contains = lambda col, contains, replace: ifelse(X[col].str.contains(contains),X[col].str.replace(replace,'',regex=False),X[col])
else_ = True

walmart_clean = (walmart
    >> mutate(Description = ifelse(X.Description.str.contains(" ; Supercenter"),
                                  X.Descrption.str.replace(';',''), X.Description))
    >> mutate(Description = X.Description.str.replace(";",","))
    >> mutate(Store_type = split_and_get('Description',',',0),
              Store_number = split_and_get('Description',',',1),
              has_gas = recode_ifelse('Description','Gas',False,1,0),
              has_diesel = recode_ifelse('Description','Gas/Diesel',False,1,0),
              has_NOP = recode_ifelse('Address_Phone','(NOP)',False,1,0),
              Address_Phone = remove_sub_if_contains('Address_Phone','NOP',',(NOP)'),
              Country = recode_ifelse('Address_Phone',',[A-Z]{2},',True,"USA","Canada"),
              State0Province = X.Address_Phone.str.extract(",([A-Z]{2})"),
              PhoneNumber = case_when((text_filter(X.Address_Phone,'\(?\d{3} ?(\)|-|_|\()? ?\d{3}( |-)?\d{4}', regex=True),
                                       X.Address_Phone.str.split(",").str.get(-1).str.replace("\(|\)| |-|_","", regex=True)),
                                      (True, "Missing/Invalid")))
)

walmart_clean

/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)
/home/wavessurfer/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/vector.py:110: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  return coalescer.lookup(np.arange(coalescer.shape[0]), min_nonna)


,Long,Lat,Description,Address_Phone,Store_type,Store_number,has_gas,has_diesel,has_NOP,Country,State0Province,PhoneNumber
0,-114.005671,51.262567,"Walmart Supercentre, #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-1295",Walmart Supercentre,#1050,0,0,0,Canada,AB,4039451295
1,-111.900542,50.577939,"Walmart Supercentre, #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",Walmart Supercentre,#3658,0,0,0,Canada,AB,4037932111
2,-114.039133,51.107253,"Walmart Supercentre, #3013,","1110 57th Ave NE,Calgary ,AB T2E 9B7,(403) 730-0990",Walmart Supercentre,#3013,0,0,1,Canada,AB,4037300990
3,-114.138488,51.040871,"Walmart Supercentre, #3009,Gas,","1212 37 St SW,Calgary ,AB T3C 1S3,(403) 242-2205",Walmart Supercentre,#3009,1,0,1,Canada,AB,4032422205
4,-114.028603,50.930551,"Walmart, #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,(403) 225-6638",Walmart,#1144,0,0,0,Canada,AB,4032256638
...,...,...,...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,"Walmart Supercenter, #4471,Gas,","2390 E Cedar St; I-80 Exit 214,Rawlins,WY,82301 ,,(307) 417-3001",Walmart Supercenter,#4471,1,0,0,USA,WY,3074173001
6812,-108.379227,43.042858,"Walmart Supercenter, #1457,","1733 N Federal Blvd,Riverton,WY,82501 ,,(307) 856-3261",Walmart Supercenter,#1457,0,0,0,USA,WY,3078563261
6813,-109.251020,41.579761,"Walmart Supercenter, #1461,","201 Gateway Blvd; I-80 Exit 102,Rock Springs,WY,82901 ,,(307) 362-1957",Walmart Supercenter,#1461,0,0,0,USA,WY,3073621957
6814,-106.940967,44.779474,"Walmart Supercenter, #1508,","1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,82801 ,(307) 674-6492",Walmart Supercenter,#1508,0,0,1,USA,WY,3076746492


In [284]:
# Write the results to a file named walmart_locations_clean.csv.  Make sure to include this file in your submission on D2L
walmart_clean.to_csv("./data/walmart_locations_clean.csv")